In [17]:
!nvidia-smi

Mon Nov 30 19:48:58 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 450.66       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     On   | 00000000:19:00.0 Off |                  Off |
| 34%   38C    P8    32W / 260W |   1202MiB / 48601MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 8000     On   | 00000000:1A:00.0 Off |                  Off |
| 46%   

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [3]:
import os.path as osp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import cm
import torch
import torch.nn as nn
import datetime
import time
%matplotlib notebook
from IPython.core.debugger import set_trace
from torch_geometric.data import Data, Dataset, InMemoryDataset, DataLoader,DataListLoader
import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU, GRU, BatchNorm1d, LayerNorm
from torch_geometric.nn import GCNConv, GENConv, DeepGCNLayer, DataParallel
import pyvista as pv
from threading import Thread
import vtk
from IPython.display import HTML, display
from pyvirtualdisplay import Display
from pathlib import Path
from torch.utils.tensorboard import SummaryWriter
np.random.seed(0)
torch.manual_seed(0)
pv.set_plot_theme("document")

In [4]:
def load_simulation_input(simulation_id, simulation_result_path = '/home/mmv664/Documents/Graph_modeling/AutoGAMMA/2_simulation_results'):
    edge_index_list = []
    pos_list = []
    elements = []
    birth_list_element = []
    birth_list_node = []
    with open('{}/{}/gamma/{}.k'.format(simulation_result_path, simulation_id, simulation_id)) as f:
        while True:
            line = next(f)
            if not line.split():
                continue
            if line.split()[0] == '*NODE':
                first = True
                while True:
                    line = next(f)
                    if line[0] == '*':
                        break
                    if line[0] == '$':
                        continue
                    text = line.split()
                    if first:
                        node_base = int(text[0])
                        first = False
                    pos_list.append([float(text[1]),float(text[2]),float(text[3])])
            if line.split()[0] == '*END':
                break  
    birth_list_node = [-1 for _ in range(len(pos_list))]
    with open('{}/{}/gamma/{}.k'.format(simulation_result_path, simulation_id, simulation_id)) as f:
        while True:
            line = next(f)
            if not line.split():
                continue
            if line.split()[0] == '*ELEMENT_SOLID':
                first = True
                while True:
                    line = next(f)
                    if line[0] == '*':
                        break
                    if line[0] == '$':
                        continue
                    text = line.split()
                    if first:
                        element_base = int(text[0])
                        first = False
                    elements.append([int(text[2])-node_base, int(text[3])-node_base, int(text[4])-node_base, int(text[5])-node_base,
                                     int(text[6])-node_base, int(text[7])-node_base, int(text[8])-node_base, int(text[9])-node_base])
                    for source_ind in range(2, 10):
                        for taget_ind in range(2, 10):
                            if source_ind == taget_ind:
                                continue
                            edge_index_list.append([int(text[source_ind])-node_base, int(text[taget_ind])-node_base])
            if line.split()[0] == '*END':
                break
    birth_list_element = [-1.0]*len(elements)
    with open('{}/{}/gamma/{}.k'.format(simulation_result_path, simulation_id, simulation_id)) as f:
        while True:
            line = next(f)
            if not line.split():
                continue
            if line.split()[0] == '*DEFINE_CURVE':
                while True:
                    line = next(f)
                    if line[0] == '*':
                        break
                    if line[0] == '$':
                        continue
                    text = line.split()
                    birth_list_element[int(float(text[1]))-element_base] = float(text[0])
            if line.split()[0] == '*END':
                break     
    
    for element, birth_element in zip(elements, birth_list_element):
        if birth_element < 0:
            continue
        for node in element:
            if (birth_list_node[node] > birth_element or 
                                        birth_list_node[node] < 0):
                                    birth_list_node[node] = birth_element

    edge_index = torch.tensor(edge_index_list ,dtype=torch.long)
    elements = torch.tensor(elements ,dtype=torch.long)
    pos = torch.tensor(pos_list ,dtype=torch.float)
    birth_element = torch.tensor(birth_list_element, dtype=torch.float)
    birth_node = torch.tensor(birth_list_node, dtype=torch.float)
    return edge_index, elements, pos, birth_element, birth_node

def load_toolpath(simulation_id, simulation_result_path = '/home/mmv664/Documents/Graph_modeling/AutoGAMMA/2_simulation_results', dt = 0.1):
    file = '{}/{}/gamma/toolpath.crs'.format(simulation_result_path, simulation_id)
    toolpath_raw=pd.read_table(file, delimiter=r"\s+",header=None, names=['time','x','y','z','state'])
    toolpath=[]
    state=[]
    time=0.0
    ind=0
    endTime = float(toolpath_raw.tail(1)['time'])
    while(time<=endTime):
        while(time>=toolpath_raw['time'][ind+1]):
            ind=ind+1
        X=toolpath_raw['x'][ind]+(toolpath_raw['x'][ind+1]-toolpath_raw['x'][ind])*(
            time-toolpath_raw['time'][ind])/(toolpath_raw['time'][ind+1]-toolpath_raw['time'][ind])
        Y=toolpath_raw['y'][ind]+(toolpath_raw['y'][ind+1]-toolpath_raw['y'][ind])*(
            time-toolpath_raw['time'][ind])/(toolpath_raw['time'][ind+1]-toolpath_raw['time'][ind])
        Z=toolpath_raw['z'][ind]+(toolpath_raw['z'][ind+1]-toolpath_raw['z'][ind])*(
            time-toolpath_raw['time'][ind])/(toolpath_raw['time'][ind+1]-toolpath_raw['time'][ind])
        toolpath.append([X,Y,Z])
        state.append(toolpath_raw['state'][ind+1])
        time = time +dt
    return toolpath, state, endTime

def load_simulation_output(simulation_id, simulation_result_path = '/home/mmv664/Documents/Graph_modeling/AutoGAMMA/2_simulation_results'):
    temperature = pd.read_csv('{}/{}/gamma/{}.csv'.format(simulation_result_path, simulation_id, simulation_id))
    temperature.drop(temperature.columns[len(temperature.columns)-1], axis=1, inplace=True)
    return torch.tensor(temperature.values, dtype=torch.float)

In [5]:
class AdditiveThermalDataset(Dataset):
    def __init__(self, root, transform=None, pre_transform=None, normalize=True):
        self.seq_len = 30 # length of each sample
        self.sample_per_simulation = 100 # number of samples in each simulation
        # self.simulation_ids = ['17', '18', '19'] # for testing
        self.simulation_ids = [x.stem for x in Path('/home/mmv664/Documents/Graph_modeling/AutoGAMMA/2_simulation_results').iterdir() if x.is_dir()]
        self.normalize = normalize
        if normalize:
            self.stats = torch.load("/home/mmv664/Documents/Graph_modeling/Graph_AM_Modeling/processed/stats")
        super(AdditiveThermalDataset, self).__init__(root, transform, pre_transform)

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return ['am_graph.simul0.data0']

    def download(self):
        pass
    
    def process(self):
        
        i = 0 # simulation_idx
        for simulation_id in self.simulation_ids:
            print("processing simulation {}!".format(simulation_id))
            j = 0 # time_idx
        
            # load simulation settings
            edge_index, elements, pos, birth_element, birth = load_simulation_input(simulation_id)
            toolpath, state, endTime = load_toolpath(simulation_id)
            
            # temperature output
            temperature = load_simulation_output(simulation_id)
            temperature = torch.clamp(temperature, 300.0, 2000.0)

            # edge feature (length of each edge)
            edge_pos = pos[edge_index.long()]
            edge_dis = ((edge_pos[:, 0, :] - edge_pos[:, 1, :])**2).sum(dim = 1)
            
            # boundary feature (distance to fixed boundary condition at z = -20)
            boundary = pos[:, 2] + 20.0

            assert (temperature.shape[1] == pos.shape[0])

            seed = np.random.permutation(temperature.shape[0]-self.seq_len)[0:self.sample_per_simulation] # the starting time of each sample 
            torch.save(seed, osp.join(self.processed_dir, 'simul{}.seed'.format(i)))
            j = 0
            for start_time in seed:
                x_data = torch.Tensor([])
                y_data = torch.Tensor([])
                birth_data = torch.Tensor([])
                for time_step in range(start_time, start_time + self.seq_len):
                    toolpath_current_step = toolpath[time_step]
                    state_current_step = state[time_step]   
                    r2 = (pos[:, 0] - toolpath_current_step[0])**2 + (pos[:, 1] - toolpath_current_step[1])**2 + (pos[:, 2] - toolpath_current_step[2])**2 + 0.1
                    if state_current_step == 1:
                        laser_feature = 1/r2
                    else:
                        laser_feature = r2 * 0.0                   

                    temp_time_step = temperature[time_step, :]
                    # temp_time_step_next = temperature[time_step + 1, :]

                    birth_time_step = (birth < time_step * 0.1).float()
                    # birth_time_step_next = (birth < (time_step + 1) * 0.1).float()

                    birth_time_element = (birth_element < time_step * 0.1).float()

                    x = torch.cat([
                    #              temp_time_step.unsqueeze(1),
                                   birth_time_step.unsqueeze(1),
                                   boundary.unsqueeze(1),
                                   laser_feature.unsqueeze(1)], dim = 1)
                    y = torch.cat([temp_time_step.unsqueeze(1),
                    #              temp_time_step_next.unsqueeze(1),
                    #              birth_time_step_next.unsqueeze(1)
                                  ], dim = 1)

                    x_data = torch.cat((x_data, x.unsqueeze(0)),0)
                    y_data = torch.cat((y_data, y.unsqueeze(0)),0)
                    birth_data = torch.cat((birth_data, birth_time_element.unsqueeze(0)),0)

                data = Data(x=x_data, y=y_data, birth_element=birth_data)
                torch.save(data, osp.join(self.processed_dir, 'am_graph.simul{}.data{}'.format(i,j)))

                j = j + 1
                if j%40 == 0:
                    print("{:.2f} %".format(j/2))

            const_data = Data(pos = pos, edge_index=edge_index.t().contiguous(), edge_attr = edge_dis, elements=elements)
            torch.save(const_data, osp.join(self.processed_dir, 'am_graph.simul{}.const'.format(i)))
            i += 1
            
    def len(self):
        return self.sample_per_simulation * len(self.simulation_ids)

    def get(self, idx):
        i = idx//self.sample_per_simulation
        j = idx%self.sample_per_simulation
        constant_data = torch.load(osp.join(self.processed_dir, 'am_graph.simul{}.const'.format(i)))
        data = torch.load(osp.join(self.processed_dir, 'am_graph.simul{}.data{}'.format(i,j)))
        data_x = data['x']
        data_x[:,:,2] = self.smooth(data['x'][:,:,2].clamp(0, 1.0), base_temp = 0.35)
        data_e = constant_data['edge_attr']
        
        data_y = self.smooth(data['y'], base_temp = 5000)
       
        if self.normalize:
            # normalize boundary feature to the range of 0-1
            data_x[:,:,1] = (data_x[:,:,1] - self.stats[1,2]) / (self.stats[1,3] - self.stats[1,2])
            # normalize laser feature to mean of zeros and std of 1
            #data_x[:,:,2] = (data_x[:,:,2] - self.stats[2,2]) / (self.stats[2,3] - self.stats[2,2])
            # nomalize edge feature to mean of zero and std of 1
            data_e = 1.0/data_e**0.5
            # normalize temperature to the range of 0-1
            data_y = (data_y - 300) / 6200 # max_temp ~6500
            #data_y = (data_y - self.stats[3,2]) / (self.stats[3,3] - self.stats[3,2])
            
        return Data(
                    x=data_x.transpose(0,1)[:,0:self.seq_len,:], 
                    y=data_y.transpose(0,1)[:,0:self.seq_len,:], 
                    pos=constant_data['pos'], 
                    edge_index=constant_data['edge_index'],
                    edge_attr=data_e.unsqueeze(1),
                    birth_element=data['birth_element'].transpose(0,1)[:,0:self.seq_len],
                    elements=constant_data['elements']
                   )
    def smooth(self, data, base_temp = 5000):
        added = (data - base_temp).clamp(min = 0) / 10
        final = data.clamp(max = base_temp) + added
        return final

In [6]:
dataset = AdditiveThermalDataset('/home/mmv664/Documents/Graph_modeling/Graph_AM_Modeling', normalize=True)

In [7]:
test_simulations = 5
#train_samples = 10000
test_start_index = (len(dataset.simulation_ids) - test_simulations)* dataset.sample_per_simulation
train_dataset = dataset[:test_start_index]
train_dataset = train_dataset.shuffle()
#train_dataset = train_dataset[:train_samples]
test_dataset = dataset[test_start_index:]
len(train_dataset), len(test_dataset)

(4000, 500)

In [8]:
batch_size = 1
#train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
#test_loader = DataLoader(test_dataset, batch_size = batch_size)
train_loader = DataListLoader(train_dataset, batch_size = batch_size, shuffle=True)
test_loader = DataListLoader(test_dataset, batch_size = batch_size)
visualization_loader = DataListLoader(train_dataset, batch_size=1)

In [9]:
# # only run for the first time when building the stats, other times stats can be just loaded from AdditiveThermalDataset

# sums, squared_sums, num_sample, min_, max_ = torch.zeros(4), torch.zeros(4), torch.zeros(4), torch.ones(4) * 1e10, torch.ones(4) * -1e10
# sums_e, squared_sums_e, num_sample_e, min_e, max_e = torch.zeros(1), torch.zeros(1), torch.zeros(1), torch.ones(1) * 1e10, torch.ones(1) * -1e10

# for data in train_dataset:
#     data_xy = torch.cat((data.x, data.y), 2)
#     data_e = data.edge_attr
#     sums += torch.sum(data_xy, dim=[0,1])
#     sums_e += torch.sum(data_e, dim=[0])
#     squared_sums += torch.sum(data_xy**2, dim=[0, 1])
#     squared_sums_e += torch.sum(data_e**2, dim=[0])
#     num_sample += data_xy.shape[0] * data_xy.shape[1]
#     num_sample_e += data_e.shape[0]
#     min_ = torch.min(min_, data_xy.view(-1,data_xy.shape[-1]).min(dim=0).values)
#     max_ = torch.max(max_, data_xy.view(-1,data_xy.shape[-1]).max(dim=0).values)
#     min_e = torch.min(min_e, data_e.min(dim=0).values)
#     max_e = torch.max(max_e, data_e.max(dim=0).values)    

# mean = sums/num_sample
# std = (squared_sums/num_sample - mean**2)**0.5
# mean_e = sums_e/num_sample_e
# std_e = (squared_sums_e/num_sample_e - mean_e**2)**0.5

# stats_xy = torch.cat((mean.unsqueeze(1),
#                   std.unsqueeze(1), 
#                   min_.unsqueeze(1),
#                   max_.unsqueeze(1)),
#                   1)
# stats_e = torch.cat((mean_e, std_e, min_e, max_e)).unsqueeze(0)
# stats = torch.cat((stats_xy, stats_e), 0)
# torch.save(stats, "processed/stats")

In [10]:
dim = 8
input_features = 3
output_features = 1
num_layers = 6
#dropout = 0.

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.node_encoder = Linear(input_features, dim)
        self.edge_encoder = Linear(1, dim)
        self.layers = torch.nn.ModuleList()
        for i in range(1, num_layers + 1):
            conv = GENConv(dim, dim, aggr='softmax',
                           t=1.0, learn_t=True, num_layers=2, norm='layer')
            norm = LayerNorm(dim, elementwise_affine=True)
            act = ReLU(inplace=True)
            layer = DeepGCNLayer(conv, norm, act, block='res+', dropout=0.1,
                                 ckpt_grad=i % 3)
            self.layers.append(layer)

        self.lin = Linear(dim, output_features)
        
        self.gru1 = torch.nn.GRU(dim, dim, num_layers=3)
        self.gru2 = torch.nn.GRU(dim, 1)
        


    def forward(self, data):
        X, Y, edge_index, edge_attr = data.x, data.y, data.edge_index, data.edge_attr
        edge_attr = self.edge_encoder(edge_attr)
        # extract features from time sequence 
        for i in range(1,X.shape[1]):
            x = X[:,i,:]
            x = self.node_encoder(x)
            
            feature = self.layers[0].conv(x, edge_index, edge_attr)
            
            for layer in self.layers[1:]:
                feature = layer(feature, edge_index, edge_attr)
                
            feature = self.layers[0].act(self.layers[0].norm(feature))
            feature = F.dropout(feature, p=0.1, training=self.training)
            
            if i == 1:
                features = feature.unsqueeze(0)
            else:
                features = torch.cat((features, feature.unsqueeze(0)),0)
        
        # initial state: node birth info
        h_0 = X[:,0,0].reshape(1,-1,1).repeat([3,1,dim]).contiguous()
        out,h = self.gru1(features,h_0)
        
        # initial state: temp
        h_0 = Y[:,0,:].unsqueeze(0).contiguous()
        out,h = self.gru2(out,h_0)
        
        return out

In [11]:
logdir='results/{}'.format(datetime.datetime.now().strftime("%Y-%m-%d--%H-%M-%S"))
description = ""

os.makedirs(logdir, exist_ok=True)
writer = SummaryWriter(logdir)
writer.add_text("Final_model", description)

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net()
model = DataParallel(model)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
%%time
for data_list in train_loader:
        predict = model(data_list)
        # mask the node not born 
#         output = data_list[0].y[:,1:,:].transpose(0,1).to(device)
#         birth = data_list[0].x[:,1:,0].transpose(0,1).to(device)
#         loss = F.mse_loss(predict[:,:,0]*birth, output[:,:,0]*birth)*birth.shape[0]*birth.shape[1]/birth.sum()
        break

In [13]:
def train(loader):
    model.train()
    loss_all = 0
    for data_list in loader:
        optimizer.zero_grad()
        predict = model(data_list)
        # mask the node not born 
        output = data_list[0].y[:,1:,:].transpose(0,1).to(device)
        #birth = data_list[0].x[:,1:,0].transpose(0,1).to(device)
        #loss = F.mse_loss(predict[:,:,0]*birth, output[:,:,0]*birth)*birth.shape[0]*birth.shape[1]/birth.sum()
        loss = F.mse_loss(predict[:,:,0], output[:,:,0])
        loss.backward()
        loss_all += len(data_list) * loss.item()
        optimizer.step()
    return loss_all / (len(loader) * loader.batch_size)

def test(loader):
    model.eval()
    loss_all = 0
    with torch.no_grad():
        for data_list in loader:
            predict = model(data_list)
             # mask the node not born
            output = data_list[0].y[:,1:,:].transpose(0,1).to(device)
            #birth = data_list[0].x[:,1:,0].transpose(0,1).to(device)
            #loss = F.mse_loss(predict[:,:,0]*birth, output[:,:,0]*birth)*birth.shape[0]*birth.shape[1]/birth.sum()
            loss = F.mse_loss(predict[:,:,0], output[:,:,0])
            loss_all += len(data_list) * loss.item()
    return loss_all / (len(loader) * loader.batch_size)

In [ ]:
his = []
# trian_loss = test(train_loader)
# test_loss = test(test_loader)
# his.append([trian_loss, test_loss])
# writer.add_scalar("train_loss", trian_loss, 0)
# writer.add_scalar("test_loss", test_loss, 0)
# print('[{}] Epoch: {:03d}, Train loss: {:.2E}, Test loss: {:.2E} '.format(
#     str(datetime.datetime.now().strftime('%H:%M:%S')), 0, trian_loss,test_loss))
for epoch in range(0, 200):
    train_loss = train(train_loader)
    test_loss = test(test_loader)
    his.append([train_loss, test_loss])
    writer.add_scalar("train_loss", train_loss, epoch)
    writer.add_scalar("test_loss", test_loss, epoch)
    print('[{}] Epoch: {:03d}, Train loss: {:.2E}, Test loss: {:.2E} '.format(
        str(datetime.datetime.now().strftime('%H:%M:%S')), epoch, train_loss,test_loss))
    torch.save(his, 'his_final.txt')
    torch.save(model.state_dict(), 'Models/RGNN_final.pt')

[00:38:05] Epoch: 000, Train loss: 2.43E-02, Test loss: 8.95E-04 
[03:03:34] Epoch: 001, Train loss: 5.53E-04, Test loss: 2.98E-04 
[05:29:12] Epoch: 002, Train loss: 2.17E-04, Test loss: 1.73E-04 
[07:54:54] Epoch: 003, Train loss: 1.42E-04, Test loss: 1.10E-04 
[10:20:50] Epoch: 004, Train loss: 8.10E-05, Test loss: 5.27E-05 
[12:46:27] Epoch: 005, Train loss: 5.24E-05, Test loss: 4.22E-05 
[15:13:19] Epoch: 006, Train loss: 4.58E-05, Test loss: 4.66E-05 
[17:39:55] Epoch: 007, Train loss: 4.23E-05, Test loss: 3.89E-05 
[20:06:09] Epoch: 008, Train loss: 3.97E-05, Test loss: 3.60E-05 
[22:32:17] Epoch: 009, Train loss: 3.80E-05, Test loss: 3.42E-05 
[00:58:11] Epoch: 010, Train loss: 3.62E-05, Test loss: 3.35E-05 
[03:24:43] Epoch: 011, Train loss: 3.48E-05, Test loss: 3.15E-05 
[05:50:23] Epoch: 012, Train loss: 3.35E-05, Test loss: 2.98E-05 
[08:16:19] Epoch: 013, Train loss: 3.23E-05, Test loss: 3.18E-05 
[10:42:32] Epoch: 014, Train loss: 3.15E-05, Test loss: 2.75E-05 
[13:08:20]

In [ ]:
his = np.array(his)
cmap1 = cm.get_cmap('Blues')(np.linspace(1, 0.5, 3))
cmap2 = cm.get_cmap('Reds')(np.linspace(1, 0.5, 3))
fig, ax = plt.subplots(1, figsize = (6,4))
ax.plot(range(his.shape[0]), his[:,0], color=cmap1[0], label='training loss')
ax.plot(range(his.shape[0]), his[:,1], color=cmap2[0], label='test loss')
#ax.set_ylim([0,0.001])
ax.set_ylabel('loss')
ax.set_xlabel('epoch')
ax.legend()
# ax.set_ylim([-1,5])
plt.tight_layout()
plt.show()

In [ ]:
torch.save(model.state_dict(), 'Models/RGNN_final.pt')

In [ ]:
path = 'Models/RGNN_final.pt'
#model = Net()
#model = DataParallel(model)
#model = model.to(device)
model.load_state_dict(torch.load(path))
model.eval()

### Visualization

In [ ]:
def display_temp_error(temp,data,time=99,zoom=3.0,show_edges=False,
                        camera_position = [(200, 100, 100),(10.0, 10.0, 0.0), (0.0, 0.0, 1.0)],
                        camera_position_zoom = [(200/3, 100/3, 100/3),(10.0, 10.0, 0.0), (0.0, 0.0, 1.0)]):
    display = Display(visible=0)
    _ = display.start()
    output_show = temp[time-1,:,0]
    range_ = [-max(-output_show.min(),output_show.max()), max(-output_show.min(),output_show.max())]
    active_elements = [element.tolist() for element, birth_time in zip(data['elements'], data['birth_element'][:,time]) if birth_time > 0.5]
    active_cells = np.array([item for sublist in active_elements for item in [8] + sublist])
    active_cell_type = np.array([vtk.VTK_HEXAHEDRON] * len(active_elements))
    points = np.array(data['pos'])
    active_grid = pv.UnstructuredGrid(active_cells, active_cell_type, points)
    active_grid.point_arrays['temp'] = np.array(output_show)
    elements = [element.tolist() for element, birth_time in zip(data['elements'], data['birth_element']) if True]
    cells = np.array([item for sublist in elements for item in [8] + sublist])
    cell_type = np.array([vtk.VTK_HEXAHEDRON] * len(elements))
    grid = pv.UnstructuredGrid(cells, cell_type, points)
    grid.point_arrays['temp'] = np.array(output_show)
    clipped = grid.clip('x', origin=(-2,0,0), invert=True)
    p = pv.Plotter(shape=(1, 1), window_size=([800, 300]),)
    p.camera_position = camera_position_zoom
    p.add_mesh(active_grid, show_edges=show_edges, scalars='temp',cmap="coolwarm",clim = range_)
    p.show()

In [ ]:
def display_temp_history(temp,data,time=99,zoom=3.0,show_edges=False,
                        camera_position = [(200, 100, 100),(10.0, 10.0, 0.0), (0.0, 0.0, 1.0)],
                        camera_position_zoom = [(200/3, 100/3, 100/3),(10.0, 10.0, 0.0), (0.0, 0.0, 1.0)]):
    display = Display(visible=0)
    _ = display.start()
    output_show = temp[time-1,:,0]
    range_ = [300, 3000]
    active_elements = [element.tolist() for element, birth_time in zip(data['elements'], data['birth_element'][:,time]) if birth_time > 0.5]
    active_cells = np.array([item for sublist in active_elements for item in [8] + sublist])
    active_cell_type = np.array([vtk.VTK_HEXAHEDRON] * len(active_elements))
    points = np.array(data['pos'])
    active_grid = pv.UnstructuredGrid(active_cells, active_cell_type, points)
    active_grid.point_arrays['temp'] = np.array(output_show)
    elements = [element.tolist() for element, birth_time in zip(data['elements'], data['birth_element']) if True]
    cells = np.array([item for sublist in elements for item in [8] + sublist])
    cell_type = np.array([vtk.VTK_HEXAHEDRON] * len(elements))
    grid = pv.UnstructuredGrid(cells, cell_type, points)
    grid.point_arrays['temp'] = np.array(output_show)
    clipped = grid.clip('x', origin=(-2,0,0), invert=True)
    p = pv.Plotter(shape=(1, 1), window_size=([800, 300]),)
    p.camera_position = camera_position_zoom
    p.add_mesh(active_grid, show_edges=show_edges, scalars='temp',cmap="coolwarm",clim = range_)
    p.show()


In [ ]:
# test_id = np.random.randint(0,len(dataset))
test_id = 0
data = [test_dataset[test_id]]
with torch.no_grad():
    prediction =  model(data).to("cpu")
data = data[0].to("cpu")
output = data.y[:,1:,:].transpose(0,1)
output = output*(5000-300) + 300
prediction = prediction*(5000-300) + 300
birth = data.x[:,1:,0].transpose(0,1).unsqueeze(2)
prediction = prediction*birth
output = output*birth
error = np.array(prediction - output)
error.shape

In [ ]:
t = 29
display_temp_history(output,data,time=t)
display_temp_history(prediction,data,time=t)
display_temp_error(prediction-output,data,time=t)

In [ ]:
data.x[:,9,2].argmax()

In [ ]:
point_id = 2959
prediction-output[:,point_id,0]
cmap1 = cm.get_cmap('Blues')(np.linspace(1, 0.5, 3))
cmap2 = cm.get_cmap('Reds')(np.linspace(1, 0.5, 3))
cmap3 = cm.get_cmap('Greens')(np.linspace(1, 0.5, 3))

fig, ax = plt.subplots(1, figsize = (6,4))
ax.plot(np.linspace(0.1, 3, 29),prediction[:,point_id,0], color=cmap1[0], label='predicted_temp')
ax.plot(np.linspace(0.1, 3, 29),output[:,point_id,0], color=cmap2[0], label='actual_temp')
ax.plot(np.linspace(0.1, 3, 29),output[:,point_id,0]-prediction[:,point_id,0], color=cmap3[0], label='error')
ax.set_ylabel('temp (K)')
ax.set_xlabel('time (s)')
ax.legend()
# ax.set_ylim([-1,5])
plt.tight_layout()
plt.show()

In [ ]:
simulation_id = '18'
start_time = 5000
seq_len = 3000

# load simulation settings
edge_index, elements, pos, birth_element, birth = load_simulation_input(simulation_id)
toolpath, state, endTime = load_toolpath(simulation_id)

# temperature output
temperature = load_simulation_output(simulation_id)
#temperature = torch.clamp(temperature, 300.0, 2000.0)

# edge feature (length of each edge)
edge_pos = pos[edge_index.long()]
edge_dis = ((edge_pos[:, 0, :] - edge_pos[:, 1, :])**2).sum(dim = 1)

# boundary feature (distance to fixed boundary condition at z = -20)
boundary = pos[:, 2] + 20.0

assert (temperature.shape[1] == pos.shape[0])

x_data = torch.Tensor([])
y_data = torch.Tensor([])
birth_data = torch.Tensor([])
for time_step in range(start_time, start_time + seq_len):
    toolpath_current_step = toolpath[time_step]
    state_current_step = state[time_step]   
    r2 = (pos[:, 0] - toolpath_current_step[0])**2 + (pos[:, 1] - toolpath_current_step[1])**2 + (pos[:, 2] - toolpath_current_step[2])**2 + 0.1
    if state_current_step == 1:
        laser_feature = 1/r2
    else:
        laser_feature = r2 * 0.0                   

    temp_time_step = temperature[time_step, :]
    # temp_time_step_next = temperature[time_step + 1, :]

    birth_time_step = (birth < time_step * 0.1).float()
    # birth_time_step_next = (birth < (time_step + 1) * 0.1).float()

    birth_time_element = (birth_element < time_step * 0.1).float()

    x = torch.cat([
    #              temp_time_step.unsqueeze(1),
                   birth_time_step.unsqueeze(1),
                   boundary.unsqueeze(1),
                   laser_feature.unsqueeze(1)], dim = 1)
    y = torch.cat([temp_time_step.unsqueeze(1),
    #              temp_time_step_next.unsqueeze(1),
    #              birth_time_step_next.unsqueeze(1)
                  ], dim = 1)

    x_data = torch.cat((x_data, x.unsqueeze(0)),0)
    y_data = torch.cat((y_data, y.unsqueeze(0)),0)
    birth_data = torch.cat((birth_data, birth_time_element.unsqueeze(0)),0)

data_temp = Data(x=x_data, y=y_data, birth_element=birth_data,
            pos = pos, edge_index=edge_index.t().contiguous(), edge_attr = edge_dis, elements=elements)

data_x = data_temp['x']
data_e = data_temp['edge_attr']
data_y = data_temp['y']
stats = torch.load("/home/mmv664/Documents/Graph_modeling/Graph_AM_Modeling/processed/stats")

# normalize boundary feature to the range of 0-1
data_x[:,:,1] = (data_x[:,:,1] - stats[1,2]) / (stats[1,3] - stats[1,2])
# normalize laser feature to mean of zeros and std of 1
data_x[:,:,2] = (data_x[:,:,2] - stats[2,2]) / (stats[2,3] - stats[2,2])
# nomalize edge feature to mean of zero and std of 1
data_e = 1.0/data_e**0.5
# normalize temperature to the range of 0-1
data_y = (data_y.clamp(300,3000) - 300) / 2700

data =  Data(
            x=data_x.transpose(0,1), 
            y=data_y.transpose(0,1), 
            pos=data_temp['pos'], 
            edge_index=data_temp['edge_index'],
            edge_attr = data_e.unsqueeze(1),
            birth_element=data_temp['birth_element'].transpose(0,1),
            elements=data_temp['elements']
           )
   

In [ ]:
data = [data]
with torch.no_grad():
    prediction =  model(data).to("cpu")
data = data[0].to("cpu")
output = data.y[:,1:,:].transpose(0,1)
output = output*(3000-300) + 300
prediction = prediction*(3000-300) + 300
birth = data.x[:,1:,0].transpose(0,1).unsqueeze(2)
prediction = prediction*birth
output = output*birth
error = np.array(prediction - output)
error.shape

In [ ]:
t = 499
display_temp_history(output,data,time=t)
display_temp_history(prediction,data,time=t)
display_temp_error(prediction-output,data,time=t)

In [ ]:
data.x[:,399,2].argmax()

In [ ]:
errors = []
point_id = 11403
cmap1 = cm.get_cmap('Blues')(np.linspace(1, 0.5, 3))
cmap2 = cm.get_cmap('Reds')(np.linspace(1, 0.5, 3))
fig, ax = plt.subplots(1, figsize = (6,4))
ax.plot(np.linspace(0.1, 300, 2999),prediction[:,point_id,0], color=cmap1[0], label='predicted_temp')
ax.plot(np.linspace(0.1, 300, 2999),output[:,point_id,0], color=cmap2[0], label='actual_temp')
error = output[:,point_id,0]-prediction[:,point_id,0]
ax.plot(np.linspace(0.1, 300, 2999),error, color=cmap3[0], label='error',linestyle='dashed')
errors.append(error.numpy())
ax.set_ylabel('temp (K)')
ax.set_xlabel('time (s)')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
point_id = 11844
cmap1 = cm.get_cmap('Blues')(np.linspace(1, 0.5, 3))
cmap2 = cm.get_cmap('Reds')(np.linspace(1, 0.5, 3))
fig, ax = plt.subplots(1, figsize = (6,4))
ax.plot(np.linspace(0.1, 300, 2999),prediction[:,point_id,0], color=cmap1[0], label='predicted_temp')
ax.plot(np.linspace(0.1, 300, 2999),output[:,point_id,0], color=cmap2[0], label='actual_temp')
error = output[:,point_id,0]-prediction[:,point_id,0]
ax.plot(np.linspace(0.1, 300, 2999),error, color=cmap3[0], label='error',linestyle='dashed')
errors.append(error.numpy())
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
point_id = 446
cmap1 = cm.get_cmap('Blues')(np.linspace(1, 0.5, 3))
cmap2 = cm.get_cmap('Reds')(np.linspace(1, 0.5, 3))
fig, ax = plt.subplots(1, figsize = (6,4))
ax.plot(np.linspace(0.1, 300, 2999),prediction[:,point_id,0], color=cmap1[0], label='predicted_temp')
ax.plot(np.linspace(0.1, 300, 2999),output[:,point_id,0], color=cmap2[0], label='actual_temp')
error = output[:,point_id,0]-prediction[:,point_id,0]
ax.plot(np.linspace(0.1, 300, 2999),error, color=cmap3[0], label='error',linestyle='dashed')
errors.append(error.numpy())
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
torch.cuda.empty_cache() 

In [ ]:
torch.save(errors,'err.txt')